# Plot layer-wise probing results on virulence prediction (7B)

In [6]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np

# Load the main results
raw_results = pd.read_csv("/workspaces/src/models/bionemo-framework/attack/analysis/virulence_results/layerwise_sweep_7b.csv")

# Fix column name (it's shuffle_labels, not shuffle)
results_no_shuffle = raw_results[raw_results["shuffle_labels"] == False]
results_shuffle = raw_results[raw_results["shuffle_labels"] == True]

# Load perplexity data for different checkpoint steps
ppl_data = []

# Step 0
ppl_df = pd.read_csv("/workspaces/BioRiskEval/attack/analysis/ppl_results/influenza_virulence_ld50_cleaned_BALB_C/evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv")
ppl_df['checkpoint_step'] = 0
ppl_data.append(ppl_df)

# Steps 100, 200, 500, 1000, 2000
for step in [100, 200, 500, 1000]:
    file_path = f"/workspaces/BioRiskEval/attack/analysis/ppl_results/influenza_virulence_ld50_cleaned_BALB_C/evo2_7b_1m_{step}_ncbi_virus_influenza_a_longest_seq_per_strain_subset/ppl_results_merged_epoch=0-step={step-1}-consumed_samples={step*8}.csv"
    ppl_df = pd.read_csv(file_path)
    ppl_df['checkpoint_step'] = step
    ppl_data.append(ppl_df)

# Combine all perplexity data
ppl_combined = pd.concat(ppl_data, ignore_index=True)

# Get unique checkpoint steps for consistent color mapping
# unique_steps = sorted(results_no_shuffle['checkpoint_step'].unique())
unique_steps = [0, 100, 200, 500, 1000]
# Use colors similar to the provided style
color_scale = ["#cea2d2",
            #    "#f9628b",
    # "#cb97cb",
    "#c88cc4",
    # "#bb9fd5",
    "#adace1",
    # "#bacff2",
    "#85bcf1",
    # "#96c8f2",
    "#a9d4f2"]
# color_scale = ["#f9628b",
#     # "#e28980",
#     "#f96e65",
#     # "#f2beb0",
#     "#f87f45",
#     # "#fa8500",
#     "#faab66",
#     # "#6866ea",
#     "#b5a3e1",
#     # "#ba6bcc",
#     "#f855b4",
#     "#d27899"
#     ]

# color_scale = [
#     # "#d0a6d5",
#     "#a95dd6",
#     # "#ecddef",
#     "#d6bee0",
#     # "#b49fd6",
#     "#a08ed1",
#     # "#9da0de",
#     "#acb2e5",
#     # "#818edb",
#     "#7fa0e4",
#     "#b9cbf1",
#     # "#d2e3f8",
#     "#83b3ed",
#     # "#88c0f2",
#     # "#9dccf2",
#     # "#b2d9f2",
# ]

color_scale = [
    "#6a66e9",
    "#faab66",
    "#fa8400",
    "#f9715d",
    "#e47492"
]


color_scale = color_scale[:len(unique_steps)]
# color_scale = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'][:len(unique_steps)]
color_map = {str(step): color_scale[i] for i, step in enumerate(unique_steps)}

# Create subplots
fig = make_subplots(
    rows=2, cols=3,
    horizontal_spacing=0.06,  # Slightly increased for larger violins
    vertical_spacing=0.2,
    column_widths=[0.33, 0.33, 0.34]  # Give slightly more space to the violin plot column
)

# Convert checkpoint_step to string for consistent handling
results_no_shuffle['checkpoint_step_str'] = results_no_shuffle['checkpoint_step'].astype(str)
results_shuffle['checkpoint_step_str'] = results_shuffle['checkpoint_step'].astype(str)
ppl_combined['checkpoint_step_str'] = ppl_combined['checkpoint_step'].astype(str)

# Figure 1: Layer vs Pearson (No Shuffle)
for step in unique_steps:
    step_data = results_no_shuffle[results_no_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/pearson'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=True
        ),
        row=1, col=1
    )

# Figure 2: Layer vs R² (No Shuffle)
for step in unique_steps:
    step_data = results_no_shuffle[results_no_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/r2'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=1, col=2
    )

# Figure 3: Layer vs Magnitude (No Shuffle)
for step in unique_steps:
    step_data = results_no_shuffle[results_no_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['magnitude/combined_mean'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=1, col=3
    )

# Figure 4: Layer vs Pearson (Shuffled)
for step in unique_steps:
    step_data = results_shuffle[results_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/pearson'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=2, col=1
    )

# Figure 5: Layer vs R² (Shuffled)
for step in unique_steps:
    step_data = results_shuffle[results_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/r2'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=2, col=2
    )

# Figure 6: Checkpoint Step vs Perplexity Distribution (Violin Plot)
# Create a separate violin plot using px.violin
violin_df = ppl_combined[ppl_combined['checkpoint_step'].isin(unique_steps)].copy()
# Convert checkpoint_step to string to ensure categorical treatment
violin_df['step_label'] = violin_df['checkpoint_step'].astype(str)

# Create violin plot using px with categorical x-axis
violin_fig = px.violin(
    violin_df,
    x='step_label',
    y='perplexity',
    box=True,
    points=False,  # Don't show individual points
    color='step_label',
    color_discrete_map={str(step): color_map[str(step)] for step in unique_steps},
    category_orders={'step_label': [str(step) for step in unique_steps]},
)

# Update violin figure layout - make traces larger and adjust properties
violin_fig.update_traces(
    showlegend=False, 
    opacity=0.6,
    width=0.8,  # Width of each violin (0-1, where 1 is full width)
    scalemode='width',  # Scale by width to make violins more prominent
)

# Add violin traces to the main subplot
for trace in violin_fig.data:  # type: ignore
    fig.add_trace(trace, row=2, col=3)

# Update axes labels
fig.update_xaxes(title_text="Layer ID", row=1, col=1)
fig.update_xaxes(title_text="Layer ID", row=1, col=2)
fig.update_xaxes(title_text="Layer ID", row=1, col=3)
fig.update_xaxes(title_text="Layer ID", row=2, col=1)
fig.update_xaxes(title_text="Layer ID", row=2, col=2)
fig.update_xaxes(
    title_text="Fine-Tuning Steps", 
    row=2, col=3,
    type='category',  # Ensure categorical treatment
    tickangle=0  # Keep labels horizontal
)

fig.update_yaxes(title_text="Pearson", range=[-0.1, 0.75], row=1, col=1)
fig.update_yaxes(title_text="R²", range=[-1.2, 0.5], row=1, col=2)
# Option 1: Clean scientific notation (1e0, 1e1, 1e2, etc.)
# fig.update_yaxes(
#     title_text="Magnitude (Combined Mean)", 
#     type="log",
#     dtick=1,  # Show ticks at each power of 10
#     tickformat=".0e",  # Clean scientific notation
#     row=1, col=3
# )

# Option 2: If you want superscript format (10⁰, 10¹, 10², etc.), uncomment below:
fig.update_yaxes(
    title_text="Feature Magnitude", 
    type="log",
    # tickvals=[0.1, 1000, 10000000, 100000000000, 10000000000000],
    tickvals = [1, 10000, 100000000, 1000000000000],
    ticktext=["10<sup>0</sup>", "10<sup>4</sup>", "10<sup>8</sup>", "10<sup>12</sup>"],
    row=1, col=3
)
fig.update_yaxes(title_text="Pearson", range=[-0.1, 0.75], row=2, col=1)
fig.update_yaxes(title_text="R²", range=[-1.2, 0.5], row=2, col=2)
fig.update_yaxes(title_text="Perplexity", row=2, col=3)  # Set range for violin plot

# Update layout with style similar to the provided example
fig.update_layout(
    height=900,  # Increased height to accommodate captions
    width=1600,
    title_font_size=18,
    title_x=0.5,  # Center the title
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="middle",
        y=1.05,
        xanchor="left",
        x=0.52,
        font=dict(size=25)  # Increase legend font size
    ),
    margin=dict(t=50, b=110, r=10, l=40),  # Increased margins for title and captions
    template="plotly_white",
    plot_bgcolor='white',
    # Use generic font families that work better with PDF export
    # Options: "Arial, sans-serif" or just "sans-serif" for better compatibility
    font=dict(family="Arial, sans-serif", size=18)
)

# Update all x-axes with consistent styling
for i in range(1, 6):  # First 5 subplots (layer plots)
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        dtick=5,  # Show ticks every 5 layers
        row=(i-1)//3 + 1,
        col=(i-1)%3 + 1,
        title_standoff=0,
        tickfont=dict(size=25),  # Increase tick label font size
        title_font=dict(size=25)  # Increase axis title font size
    )

# Handle the 6th subplot (violin plot) separately - it's categorical
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    row=2,
    col=3,
    title_standoff=0,
    tickfont=dict(size=25),  # Increase tick label font size
    title_font=dict(size=25)  # Increase axis title font size
)

# Update all y-axes with consistent styling
for i in range(1, 7):
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        zerolinecolor='lightgrey',
        row=(i-1)//3 + 1,
        col=(i-1)%3 + 1,
        title_standoff=0,
        tickfont=dict(size=25),  # Increase tick label font size
        title_font=dict(size=25)  # Increase axis title font size
    )

# Add captions to subplots
# Caption for Fig1 and Fig2 (positioned between them)
fig.add_annotation(
    text="(a) Layer-wise probing performance (no label shuffling)",
    xref="paper", yref="paper",
    x=0.33,  # Middle of first two columns
    y=0.46,  # Below first row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Caption for Fig3
fig.add_annotation(
    text="(b) Layer-wise feature magnitude",
    xref="paper", yref="paper",
    x=0.84,  # Third column position
    y=0.46,  # Below first row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Caption for Fig4 and Fig5 (positioned between them)
fig.add_annotation(
    text="(c) Layer-wise probing performance (with label shuffling)",
    xref="paper", yref="paper",
    x=0.33,  # Middle of first two columns
    y=-0.15,  # Below second row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Caption for Fig6
fig.add_annotation(
    text="(d) Perplexity distribution",
    xref="paper", yref="paper",
    x=0.84,  # Third column position
    y=-0.15,  # Below second row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Show the figure
fig.show()

# Option 2: If fonts still don't render correctly, try using SVG first then convert
fig.write_image("layerwise_sweep_7b.svg")
# Then convert SVG to PDF using external tools if needed

# convert svg to pdf
import subprocess
subprocess.run(["inkscape", "layerwise_sweep_7b.svg", "--export-pdf=layerwise_sweep_7b.pdf"])



/tmp/ipykernel_3854359/2544819397.py:101: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3854359/2544819397.py:102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



** Message: 03:27:37.637: Invalid glyph found, continuing...
** Message: 03:27:37.637: Invalid glyph found, continuing...
** Message: 03:27:37.637: Invalid glyph found, continuing...
** Message: 03:27:37.637: Invalid glyph found, continuing...
** Message: 03:27:37.637: Invalid glyph found, continuing...
** Message: 03:27:37.637: Invalid glyph found, continuing...
** Message: 03:27:37.637: Invalid glyph found, continuing...
** Message: 03:27:37.637: Invalid glyph found, continuing...


CompletedProcess(args=['inkscape', 'layerwise_sweep_7b.svg', '--export-pdf=layerwise_sweep_7b.pdf'], returncode=0)

# Plot layer-wise probing results on virulence prediction (40B)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np

# Load the main results
raw_results = pd.read_csv("/workspaces/src/models/bionemo-framework/attack/analysis/virulence_results/layerwise_sweep_40b.csv")

# Fix column name (it's shuffle_labels, not shuffle)
results_no_shuffle = raw_results[raw_results["shuffle_labels"] == False]
results_shuffle = raw_results[raw_results["shuffle_labels"] == True]

# Load perplexity data for different checkpoint steps
ppl_data = []

# Step 0
ppl_df = pd.read_csv("/workspaces/BioRiskEval/attack/analysis/ppl_results/influenza_virulence_ld50_cleaned_BALB_C/evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv")
ppl_df['checkpoint_step'] = 0
ppl_data.append(ppl_df)

# # Steps 100, 200, 500, 1000, 2000
# for step in [100, 200, 500, 1000]:
#     file_path = f"/workspaces/BioRiskEval/attack/analysis/ppl_results/influenza_virulence_ld50_cleaned_BALB_C/evo2_7b_1m_{step}_ncbi_virus_influenza_a_longest_seq_per_strain_subset/ppl_results_merged_epoch=0-step={step-1}-consumed_samples={step*8}.csv"
#     ppl_df = pd.read_csv(file_path)
#     ppl_df['checkpoint_step'] = step
#     ppl_data.append(ppl_df)

# Combine all perplexity data
ppl_combined = pd.concat(ppl_data, ignore_index=True)

# Get unique checkpoint steps for consistent color mapping
# unique_steps = sorted(results_no_shuffle['checkpoint_step'].unique())
unique_steps = [0]
# Use colors similar to the provided style

color_scale = [
    "#6a66e9",
    "#faab66",
    "#fa8400",
    "#f9715d",
    "#e47492"
]


color_scale = color_scale[:len(unique_steps)]
# color_scale = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'][:len(unique_steps)]
color_map = {str(step): color_scale[i] for i, step in enumerate(unique_steps)}

# Create subplots
fig = make_subplots(
    rows=2, cols=3,
    horizontal_spacing=0.06,  # Slightly increased for larger violins
    vertical_spacing=0.2,
    column_widths=[0.33, 0.33, 0.34]  # Give slightly more space to the violin plot column
)

# Convert checkpoint_step to string for consistent handling
results_no_shuffle['checkpoint_step_str'] = results_no_shuffle['checkpoint_step'].astype(str)
results_shuffle['checkpoint_step_str'] = results_shuffle['checkpoint_step'].astype(str)
ppl_combined['checkpoint_step_str'] = ppl_combined['checkpoint_step'].astype(str)

# Figure 1: Layer vs Pearson (No Shuffle)
for step in unique_steps:
    step_data = results_no_shuffle[results_no_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/pearson'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=True
        ),
        row=1, col=1
    )

# Figure 2: Layer vs R² (No Shuffle)
for step in unique_steps:
    step_data = results_no_shuffle[results_no_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/r2'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=1, col=2
    )

# Figure 3: Layer vs Magnitude (No Shuffle)
for step in unique_steps:
    step_data = results_no_shuffle[results_no_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['magnitude/combined_mean'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=1, col=3
    )

# Figure 4: Layer vs Pearson (Shuffled)
for step in unique_steps:
    step_data = results_shuffle[results_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/pearson'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=2, col=1
    )

# Figure 5: Layer vs R² (Shuffled)
for step in unique_steps:
    step_data = results_shuffle[results_shuffle['checkpoint_step'] == step]
    fig.add_trace(
        go.Scatter(
            x=step_data['layer'],
            y=step_data['test/r2'],
            mode='lines+markers',
            name='Base' if step == 0 else f'{step} steps',
            line=dict(color=color_map[str(step)]),
            marker=dict(color=color_map[str(step)]),
            legendgroup=f'step_{step}',
            showlegend=False
        ),
        row=2, col=2
    )

# Figure 6: Checkpoint Step vs Perplexity Distribution (Violin Plot)
# Create a separate violin plot using px.violin
violin_df = ppl_combined[ppl_combined['checkpoint_step'].isin(unique_steps)].copy()
# Convert checkpoint_step to string to ensure categorical treatment
violin_df['step_label'] = violin_df['checkpoint_step'].astype(str)

# Create violin plot using px with categorical x-axis
violin_fig = px.violin(
    violin_df,
    x='step_label',
    y='perplexity',
    box=True,
    points=False,  # Don't show individual points
    color='step_label',
    color_discrete_map={str(step): color_map[str(step)] for step in unique_steps},
    category_orders={'step_label': [str(step) for step in unique_steps]},
)

# Update violin figure layout - make traces larger and adjust properties
violin_fig.update_traces(
    showlegend=False, 
    opacity=0.6,
    width=0.8,  # Width of each violin (0-1, where 1 is full width)
    scalemode='width',  # Scale by width to make violins more prominent
)

# Add violin traces to the main subplot
for trace in violin_fig.data:  # type: ignore
    fig.add_trace(trace, row=2, col=3)

# Update axes labels
fig.update_xaxes(title_text="Layer ID", row=1, col=1)
fig.update_xaxes(title_text="Layer ID", row=1, col=2)
fig.update_xaxes(title_text="Layer ID", row=1, col=3)
fig.update_xaxes(title_text="Layer ID", row=2, col=1)
fig.update_xaxes(title_text="Layer ID", row=2, col=2)
fig.update_xaxes(
    title_text="Fine-Tuning Steps", 
    row=2, col=3,
    type='category',  # Ensure categorical treatment
    tickangle=0  # Keep labels horizontal
)

fig.update_yaxes(title_text="Pearson", range=[-0.1, 0.75], row=1, col=1)
fig.update_yaxes(title_text="R²", range=[-1.2, 0.5], row=1, col=2)
# Option 1: Clean scientific notation (1e0, 1e1, 1e2, etc.)
# fig.update_yaxes(
#     title_text="Magnitude (Combined Mean)", 
#     type="log",
#     dtick=1,  # Show ticks at each power of 10
#     tickformat=".0e",  # Clean scientific notation
#     row=1, col=3
# )

# Option 2: If you want superscript format (10⁰, 10¹, 10², etc.), uncomment below:
fig.update_yaxes(
    title_text="Feature Magnitude", 
    type="log",
    # tickvals=[0.1, 1000, 10000000, 100000000000, 10000000000000],
    tickvals = [1, 10000, 100000000, 1000000000000],
    ticktext=["10<sup>0</sup>", "10<sup>4</sup>", "10<sup>8</sup>", "10<sup>12</sup>"],
    row=1, col=3
)
fig.update_yaxes(title_text="Pearson", range=[-0.1, 0.75], row=2, col=1)
fig.update_yaxes(title_text="R²", range=[-1.2, 0.5], row=2, col=2)
fig.update_yaxes(title_text="Perplexity", row=2, col=3)  # Set range for violin plot

# Update layout with style similar to the provided example
fig.update_layout(
    height=900,  # Increased height to accommodate captions
    width=1600,
    title_font_size=18,
    title_x=0.5,  # Center the title
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="middle",
        y=1.05,
        xanchor="left",
        x=0.52,
        font=dict(size=25)  # Increase legend font size
    ),
    margin=dict(t=50, b=110, r=10, l=40),  # Increased margins for title and captions
    template="plotly_white",
    plot_bgcolor='white',
    # Use generic font families that work better with PDF export
    # Options: "Arial, sans-serif" or just "sans-serif" for better compatibility
    font=dict(family="Arial, sans-serif", size=18)
)

# Update all x-axes with consistent styling
for i in range(1, 6):  # First 5 subplots (layer plots)
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        dtick=5,  # Show ticks every 5 layers
        row=(i-1)//3 + 1,
        col=(i-1)%3 + 1,
        title_standoff=0,
        tickfont=dict(size=25),  # Increase tick label font size
        title_font=dict(size=25)  # Increase axis title font size
    )

# Handle the 6th subplot (violin plot) separately - it's categorical
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    row=2,
    col=3,
    title_standoff=0,
    tickfont=dict(size=25),  # Increase tick label font size
    title_font=dict(size=25)  # Increase axis title font size
)

# Update all y-axes with consistent styling
for i in range(1, 7):
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        zerolinecolor='lightgrey',
        row=(i-1)//3 + 1,
        col=(i-1)%3 + 1,
        title_standoff=0,
        tickfont=dict(size=25),  # Increase tick label font size
        title_font=dict(size=25)  # Increase axis title font size
    )

# Add captions to subplots
# Caption for Fig1 and Fig2 (positioned between them)
fig.add_annotation(
    text="(a) Layer-wise probing performance (no label shuffling)",
    xref="paper", yref="paper",
    x=0.33,  # Middle of first two columns
    y=0.46,  # Below first row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Caption for Fig3
fig.add_annotation(
    text="(b) Layer-wise feature magnitude",
    xref="paper", yref="paper",
    x=0.84,  # Third column position
    y=0.46,  # Below first row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Caption for Fig4 and Fig5 (positioned between them)
fig.add_annotation(
    text="(c) Layer-wise probing performance (with label shuffling)",
    xref="paper", yref="paper",
    x=0.33,  # Middle of first two columns
    y=-0.15,  # Below second row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Caption for Fig6
fig.add_annotation(
    text="(d) Perplexity distribution",
    xref="paper", yref="paper",
    x=0.84,  # Third column position
    y=-0.15,  # Below second row
    showarrow=False,
    font=dict(family="Times, serif", size=30),  # More compatible with PDF export
    xanchor="center"
)

# Show the figure
fig.show()

# Option 2: If fonts still don't render correctly, try using SVG first then convert
fig.write_image("layerwise_sweep_7b.svg")
# Then convert SVG to PDF using external tools if needed

# convert svg to pdf
import subprocess
subprocess.run(["inkscape", "layerwise_sweep_7b.svg", "--export-pdf=layerwise_sweep_7b.pdf"])



# Plot Perplexity distribution for Inter-species generalization

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path

# Define base directories
alphapapillomavirus_dir = Path("/workspaces/BioRiskEval/attack/analysis/ppl_results/ncbi_downloads_alphapapillomavirus_6")
simplexvirus_dir = Path("/workspaces/BioRiskEval/attack/analysis/ppl_results/ncbi_downloads_simplexvirus_human_alpha2")
imgpr_dir = Path("/workspaces/BioRiskEval/attack/analysis/ppl_results/imgpr")

# Define consistent colors for traces
# trace_colors = {
#     '0': '#3498db',  # blue
#     '25': '#e74c3c',  # red
#     '50': '#2ecc71',  # green
#     '100': '#f39c12',  # orange
#     '200': '#9b59b6', # purple
# }

trace_colors = {
    "0": "#6a66e9",
    "25": "#faab66",
    "50": "#fa8400",
    "100": "#f9715d",
    "200": "#e47492"
}

# Function to load and process CSV files
def load_perplexity_data(csv_files_dict, base_dir):
    perplexity_results = []
    stats_summary = []
    
    for trace_name, csv_file in csv_files_dict.items():
        file_path = base_dir / csv_file
        if not file_path.exists():
            print(f"Warning: File not found - {file_path}")
            continue
            
        df = pd.read_csv(file_path)
        perplexity_values = df['perplexity'].values
        
        # Add all perplexity values with their trace name
        for ppl in perplexity_values:
            perplexity_results.append({
                'trace': trace_name,
                'perplexity': ppl
            })
        
        # Calculate statistics
        stats = {
            'Model': trace_name,
            'Count': len(perplexity_values),
            'Mean': np.mean(perplexity_values),
            'Median': np.median(perplexity_values),
            'Std': np.std(perplexity_values),
            'Min': np.min(perplexity_values),
            'Max': np.max(perplexity_values),
            'Q1': np.percentile(perplexity_values, 25),
            'Q3': np.percentile(perplexity_values, 75)
        }
        stats_summary.append(stats)
        
        print(f"Loaded {trace_name}: {len(df)} samples")
        print(f"  - Mean: {stats['Mean']:.4f}, Median: {stats['Median']:.4f}")
        print(f"  - Range: [{stats['Min']:.4f}, {stats['Max']:.4f}]")
        print()
    
    return pd.DataFrame(perplexity_results), stats_summary

# Load data for alphapapillomavirus
print("Loading Alphapapillomavirus data...")
alphapapillomavirus_files = {
    "0": "evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv",
    "25": "evo2_7b_1m_25_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=24-consumed_samples=200.csv",
    "50": "evo2_7b_1m_50_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=49-consumed_samples=400.csv",
    "100": "evo2_7b_1m_100_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=99-consumed_samples=800.csv",
    "200": "evo2_7b_1m_200_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=199-consumed_samples=1600.csv",
    "500": "evo2_7b_1m_500_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=499-consumed_samples=4000.csv",
    # "1000": "evo2_7b_1m_1000_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=999-consumed_samples=8000.csv",
}
df_alphapapillomavirus, stats_alpha = load_perplexity_data(alphapapillomavirus_files, alphapapillomavirus_dir)

# Load data for simplexvirus (only has 0, 100, 200)
print("\nLoading Simplexvirus data...")
simplexvirus_files = {
    "0": "evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv",
    "25": "evo2_7b_1m_25_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=24-consumed_samples=200.csv",
    "50": "evo2_7b_1m_50_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=49-consumed_samples=400.csv",
    "100": "evo2_7b_1m_100_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=99-consumed_samples=800.csv",
    "200": "evo2_7b_1m_200_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=199-consumed_samples=1600.csv",
}
df_simplexvirus, stats_simplex = load_perplexity_data(simplexvirus_files, simplexvirus_dir)

# Load data for IMGPR 0
print("\nLoading IMGPR baseline data...")
imgpr_files = {
    "0": "evo2_7b_1m/ppl_results_imgpr_len_200000_sampled_5000_nemo2_evo2_7b_1m.csv"
}
df_imgpr, stats_imgpr = load_perplexity_data(imgpr_files, imgpr_dir)

# Create subplots with column widths (without titles - we'll add them as annotations at the bottom)
fig = make_subplots(
    rows=1, cols=3,
    column_widths=[0.42, 0.42, 0.16],
    horizontal_spacing=0.03
)

# Function to create violin plots using px.violin
def create_violin_plots(df, col, trace_order, showlegend=False):
    # Filter data to include only the traces we want
    filtered_df = df[df['trace'].isin(trace_order)].copy()
    
    # Create categorical ordering
    filtered_df['trace'] = pd.Categorical(filtered_df['trace'], categories=trace_order, ordered=True)
    
    # Create violin plot using px
    violin_fig = px.violin(
        filtered_df,
        x='trace',
        y='perplexity',
        box=True,
        points=False,  # Don't show individual points
        color='trace',
        color_discrete_map=trace_colors,
        category_orders={'trace': trace_order}
    )
    
    # Update traces
    violin_fig.update_traces(
        showlegend=showlegend,
        opacity=0.7,
        width=0.8,
        scalemode='width'
    )
    
    # Add traces to main figure
    for trace in violin_fig.data:
        fig.add_trace(trace, row=1, col=col)

# Add traces for each subplot
print("\nCreating violin plots...")

# Subplot 1: Alphapapillomavirus
trace_order_alpha = ["0","25", "50", "100", "200"]
create_violin_plots(df_alphapapillomavirus, 1, trace_order_alpha, showlegend=False)

# Subplot 2: Simplexvirus
trace_order_simplex = ["0", "25", "50", "100", "200"]
create_violin_plots(df_simplexvirus, 2, trace_order_simplex, showlegend=False)

# Subplot 3: IMGPR
create_violin_plots(df_imgpr, 3, ["0"], showlegend=False)

# Update layout
fig.update_layout(
    template="plotly_white",
    width=1600,
    height=350,  # Increased height to accommodate bottom titles
    font=dict(size=12),
    showlegend=False,  # Hide legend since we're showing labels on x-axis
    plot_bgcolor='white',
    margin=dict(t=10, b=110, l=60, r=30)  # Increased bottom margin for titles
)

# Add subplot titles as annotations at the bottom
# Title for subplot 1 (Alphapapillomavirus)
fig.add_annotation(
    text="(a) Alphapapillomavirus 6",
    xref="paper", yref="paper",
    x=0.19,  # Center of first subplot
    y=-0.45,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)

# Title for subplot 2 (Simplexvirus)
fig.add_annotation(
    text="(b) Herpes Simplex Virus 2",
    xref="paper", yref="paper",
    x=0.615,  # Center of second subplot
    y=-0.45,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)

# Title for subplot 3 (IMGPR)
fig.add_annotation(
    text="(c) IMG/PR",
    xref="paper", yref="paper",
    x=0.917,  # Center of third subplot
    y=-0.45,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)

# Update axes for all subplots
y_min = 1.0  # Set minimum to 1.0 since perplexity cannot be less than 1
y_max = 5.5

for i in range(1, 4):
    # Update x-axes
    fig.update_xaxes(
        title_text="Fine-tuning Steps",
        showticklabels=True,  # Show tick labels with step numbers
        showline=True,
        linecolor='black',
        mirror=True,
        ticks='outside',
        tickangle=0,  # Keep labels horizontal
        tickfont=dict(size=25, family="Arial"),  # Set font size for x-axis labels
        title_font=dict(size=25, family="Arial"),  # Set font size for x-axis title
        row=1,
        col=i,
        title_standoff=0
    )
    
    # Update y-axes
    fig.update_yaxes(
        range=[y_min, y_max],
        gridcolor='lightgrey',
        zerolinecolor='lightgrey',
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        title_text="Perplexity" if i == 1 else "",
        row=1,
        col=i,
        tickfont=dict(size=25, family="Arial"),
        title_font=dict(size=25, family="Arial"),
        title_standoff=3
    )

# Show the plot
fig.show()

# Save outputs
out_file_svg = "ppl_violin_interspecies.svg"   
out_file_pdf = "ppl_violin_interspecies.pdf"

fig.write_image(out_file_svg)

# convert svg to pdf
import subprocess
subprocess.run(["inkscape", out_file_svg, "--export-pdf=" + out_file_pdf])

print(f"\nPlots saved as:")
print(f"  - {out_file_pdf}")

# Display statistics summary
print("\n" + "="*80)
print("Statistical Summary - Alphapapillomavirus:")
print("="*80)
stats_df_alpha = pd.DataFrame(stats_alpha)
print(stats_df_alpha.to_string(index=False, float_format=lambda x: f'{x:.4f}'))

print("\n" + "="*80)
print("Statistical Summary - Simplexvirus:")
print("="*80)
stats_df_simplex = pd.DataFrame(stats_simplex)
print(stats_df_simplex.to_string(index=False, float_format=lambda x: f'{x:.4f}'))

print("\n" + "="*80)
print("Statistical Summary - IMGPR:")
print("="*80)
stats_df_imgpr = pd.DataFrame(stats_imgpr)
# print(stats_df_imgpr.to_string(index=False, float_format=lambda x: f'{x:.4f})


Loading Alphapapillomavirus data...
Loaded 0: 569 samples
  - Mean: 3.6993, Median: 3.7258
  - Range: [3.2147, 4.6077]

Loaded 25: 569 samples
  - Mean: 2.9896, Median: 2.9420
  - Range: [2.2491, 4.5686]

Loaded 50: 569 samples
  - Mean: 2.7537, Median: 2.6795
  - Range: [2.0901, 4.8700]

Loaded 100: 569 samples
  - Mean: 2.7041, Median: 2.5951
  - Range: [1.5417, 4.9254]

Loaded 200: 569 samples
  - Mean: 2.6875, Median: 2.5663
  - Range: [1.3676, 5.0100]

Loaded 500: 569 samples
  - Mean: 2.6575, Median: 2.5230
  - Range: [1.2944, 4.9823]


Loading Simplexvirus data...
Loaded 0: 1287 samples
  - Mean: 3.1797, Median: 3.1958
  - Range: [1.4834, 4.0177]

Loaded 25: 1287 samples
  - Mean: 2.5720, Median: 2.4715
  - Range: [1.4050, 3.7743]

Loaded 50: 1287 samples
  - Mean: 2.4150, Median: 2.2802
  - Range: [1.3880, 3.7060]

Loaded 100: 1287 samples
  - Mean: 2.3920, Median: 2.2700
  - Range: [1.3827, 3.6702]

Loaded 200: 1287 samples
  - Mean: 2.4373, Median: 2.3388
  - Range: [1.3861, 


Plots saved as:
  - ppl_violin_interspecies.pdf

Statistical Summary - Alphapapillomavirus:
Model  Count   Mean  Median    Std    Min    Max     Q1     Q3
    0    569 3.6993  3.7258 0.1351 3.2147 4.6077 3.5959 3.7771
   25    569 2.9896  2.9420 0.4228 2.2491 4.5686 2.6264 3.3529
   50    569 2.7537  2.6795 0.4564 2.0901 4.8700 2.3815 3.0780
  100    569 2.7041  2.5951 0.4448 1.5417 4.9254 2.3733 2.9612
  200    569 2.6875  2.5663 0.4395 1.3676 5.0100 2.3705 2.8527
  500    569 2.6575  2.5230 0.4521 1.2944 4.9823 2.3561 2.8113

Statistical Summary - Simplexvirus:
Model  Count   Mean  Median    Std    Min    Max     Q1     Q3
    0   1287 3.1797  3.1958 0.2749 1.4834 4.0177 2.9484 3.4491
   25   1287 2.5720  2.4715 0.4644 1.4050 3.7743 2.4003 2.8251
   50   1287 2.4150  2.2802 0.4598 1.3880 3.7060 2.2436 2.6128
  100   1287 2.3920  2.2700 0.4319 1.3827 3.6702 2.2285 2.5511
  200   1287 2.4373  2.3388 0.4377 1.3861 3.7236 2.2902 2.6637

Statistical Summary - IMGPR:


# Plot Perplexity Distribution (New) For both inter-species generalization

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path

# Define base directories
alphapapillomavirus_dir = Path("/workspaces/BioRiskEval/attack/analysis/ppl_results/ncbi_downloads_alphapapillomavirus_6")
simplexvirus_dir = Path("/workspaces/BioRiskEval/attack/analysis/ppl_results/ncbi_downloads_simplexvirus_human_alpha2")
imgpr_dir = Path("/workspaces/BioRiskEval/attack/analysis/ppl_results/imgpr")

# Define consistent colors for traces
# trace_colors = {
#     '0': '#3498db',  # blue
#     '25': '#e74c3c',  # red
#     '50': '#2ecc71',  # green
#     '100': '#f39c12',  # orange
#     '200': '#9b59b6', # purple
# }

trace_colors = {
    "0": "#6a66e9",
    "25": "#faab66",
    "50": "#fa8400",
    "100": "#f9715d",
    "200": "#e47492"
}

# Function to load and process CSV files
def load_perplexity_data(csv_files_dict, base_dir):
    perplexity_results = []
    stats_summary = []
    
    for trace_name, csv_file in csv_files_dict.items():
        file_path = base_dir / csv_file
        if not file_path.exists():
            print(f"Warning: File not found - {file_path}")
            continue
            
        df = pd.read_csv(file_path)
        perplexity_values = df['perplexity'].values
        
        # Add all perplexity values with their trace name
        for ppl in perplexity_values:
            perplexity_results.append({
                'trace': trace_name,
                'perplexity': ppl
            })
        
        # Calculate statistics
        stats = {
            'Model': trace_name,
            'Count': len(perplexity_values),
            'Mean': np.mean(perplexity_values),
            'Median': np.median(perplexity_values),
            'Std': np.std(perplexity_values),
            'Min': np.min(perplexity_values),
            'Max': np.max(perplexity_values),
            'Q1': np.percentile(perplexity_values, 25),
            'Q3': np.percentile(perplexity_values, 75)
        }
        stats_summary.append(stats)
        
        print(f"Loaded {trace_name}: {len(df)} samples")
        print(f"  - Mean: {stats['Mean']:.4f}, Median: {stats['Median']:.4f}")
        print(f"  - Range: [{stats['Min']:.4f}, {stats['Max']:.4f}]")
        print()
    
    return pd.DataFrame(perplexity_results), stats_summary

# Load data for alphapapillomavirus
print("Loading Alphapapillomavirus data...")
alphapapillomavirus_files = {
    "0": "evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv",
    "25": "evo2_7b_1m_25_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=24-consumed_samples=200.csv",
    "50": "evo2_7b_1m_50_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=49-consumed_samples=400.csv",
    "100": "evo2_7b_1m_100_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=99-consumed_samples=800.csv",
    "200": "evo2_7b_1m_200_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=199-consumed_samples=1600.csv",
    "500": "evo2_7b_1m_500_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=499-consumed_samples=4000.csv",
    # "1000": "evo2_7b_1m_1000_alphapapillomavirus_exclude6/ppl_results_merged_epoch=0-step=999-consumed_samples=8000.csv",
}
df_alphapapillomavirus, stats_alpha = load_perplexity_data(alphapapillomavirus_files, alphapapillomavirus_dir)

# Load data for simplexvirus (only has 0, 100, 200)
print("\nLoading Simplexvirus data...")
simplexvirus_files = {
    "0": "evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv",
    "25": "evo2_7b_1m_25_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=24-consumed_samples=200.csv",
    "50": "evo2_7b_1m_50_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=49-consumed_samples=400.csv",
    "100": "evo2_7b_1m_100_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=99-consumed_samples=800.csv",
    "200": "evo2_7b_1m_200_simplexvirus_exclude_human_alpha2/ppl_results_merged_epoch=0-step=199-consumed_samples=1600.csv",
}
df_simplexvirus, stats_simplex = load_perplexity_data(simplexvirus_files, simplexvirus_dir)

# Load data for IMGPR 0
print("\nLoading IMGPR baseline data...")
imgpr_files = {
    "0": "evo2_7b_1m/ppl_results_imgpr_len_200000_sampled_5000_nemo2_evo2_7b_1m.csv"
}
df_imgpr, stats_imgpr = load_perplexity_data(imgpr_files, imgpr_dir)

# Create subplots with column widths (without titles - we'll add them as annotations at the bottom)
fig = make_subplots(
    rows=1, cols=3,
    column_widths=[0.42, 0.42, 0.16],
    horizontal_spacing=0.03
)

# Function to create violin plots using px.violin
def create_violin_plots(df, col, trace_order, showlegend=False):
    # Filter data to include only the traces we want
    filtered_df = df[df['trace'].isin(trace_order)].copy()
    
    # Create categorical ordering
    filtered_df['trace'] = pd.Categorical(filtered_df['trace'], categories=trace_order, ordered=True)
    
    # Create violin plot using px
    violin_fig = px.violin(
        filtered_df,
        x='trace',
        y='perplexity',
        box=True,
        points=False,  # Don't show individual points
        color='trace',
        color_discrete_map=trace_colors,
        category_orders={'trace': trace_order}
    )
    
    # Update traces
    violin_fig.update_traces(
        showlegend=showlegend,
        opacity=0.7,
        width=0.8,
        scalemode='width'
    )
    
    # Add traces to main figure
    for trace in violin_fig.data:
        fig.add_trace(trace, row=1, col=col)

# Add traces for each subplot
print("\nCreating violin plots...")

# Subplot 1: Alphapapillomavirus
trace_order_alpha = ["0","25", "50", "100", "200"]
create_violin_plots(df_alphapapillomavirus, 1, trace_order_alpha, showlegend=False)

# Subplot 2: Simplexvirus
trace_order_simplex = ["0", "25", "50", "100", "200"]
create_violin_plots(df_simplexvirus, 2, trace_order_simplex, showlegend=False)

# Subplot 3: IMGPR
create_violin_plots(df_imgpr, 3, ["0"], showlegend=False)

# Update layout
fig.update_layout(
    template="plotly_white",
    width=1600,
    height=350,  # Increased height to accommodate bottom titles
    font=dict(size=12),
    showlegend=False,  # Hide legend since we're showing labels on x-axis
    plot_bgcolor='white',
    margin=dict(t=40, b=15, l=60, r=30)  # Increased bottom margin for titles
)

# Add subplot titles as annotations at the bottom
# Title for subplot 1 (Alphapapillomavirus)
fig.add_annotation(
    text="Alphapapillomavirus 6",
    xref="paper", yref="paper",
    x=0.195,  # Center of first subplot
    y=1.17,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)

# Title for subplot 2 (Simplexvirus)
fig.add_annotation(
    text="Herpes Simplex Virus 2",
    xref="paper", yref="paper",
    x=0.62,  # Center of second subplot
    y=1.17,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)

# Title for subplot 3 (IMGPR)
fig.add_annotation(
    text="IMG/PR",
    xref="paper", yref="paper",
    x=0.923,  # Center of third subplot
    y=1.17,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)

# Update axes for all subplots
y_min = 0.9
y_max = 5.5

for i in range(1, 4):
    # Update x-axes
    fig.update_xaxes(
        title_text="Fine-tuning Steps",
        showticklabels=True,  # Show tick labels with step numbers
        showline=True,
        linecolor='black',
        mirror=True,
        ticks='outside',
        tickangle=0,  # Keep labels horizontal
        tickfont=dict(size=25, family="Arial"),  # Set font size for x-axis labels
        title_font=dict(size=25, family="Arial"),  # Set font size for x-axis title
        row=1,
        col=i,
        title_standoff=3
    )
    
    # Update y-axes
    fig.update_yaxes(
        range=[y_min, y_max],
        gridcolor='lightgrey',
        zerolinecolor='lightgrey',
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        title_text="Perplexity" if i == 1 else "",
        row=1,
        col=i,
        tickfont=dict(size=25, family="Arial"),
        title_font=dict(size=25, family="Arial"),
        title_standoff=3
    )

# Show the plot
fig.show()

# Save outputs
out_file_svg = "ppl_violin_interspecies.svg"   
out_file_pdf = "ppl_violin_interspecies.pdf"

fig.write_image(out_file_svg)

# convert svg to pdf
import subprocess
subprocess.run(["inkscape", out_file_svg, "--export-pdf=" + out_file_pdf])

print(f"\nPlots saved as:")
print(f"  - {out_file_pdf}")

# Display statistics summary
print("\n" + "="*80)
print("Statistical Summary - Alphapapillomavirus:")
print("="*80)
stats_df_alpha = pd.DataFrame(stats_alpha)
print(stats_df_alpha.to_string(index=False, float_format=lambda x: f'{x:.4f}'))

print("\n" + "="*80)
print("Statistical Summary - Simplexvirus:")
print("="*80)
stats_df_simplex = pd.DataFrame(stats_simplex)
print(stats_df_simplex.to_string(index=False, float_format=lambda x: f'{x:.4f}'))

print("\n" + "="*80)
print("Statistical Summary - IMGPR:")
print("="*80)
stats_df_imgpr = pd.DataFrame(stats_imgpr)
# print(stats_df_imgpr.to_string(index=False, float_format=lambda x: f'{x:.4f})


Loading Alphapapillomavirus data...
Loaded 0: 569 samples
  - Mean: 3.6993, Median: 3.7258
  - Range: [3.2147, 4.6077]

Loaded 25: 569 samples
  - Mean: 2.9896, Median: 2.9420
  - Range: [2.2491, 4.5686]

Loaded 50: 569 samples
  - Mean: 2.7537, Median: 2.6795
  - Range: [2.0901, 4.8700]

Loaded 100: 569 samples
  - Mean: 2.7041, Median: 2.5951
  - Range: [1.5417, 4.9254]

Loaded 200: 569 samples
  - Mean: 2.6875, Median: 2.5663
  - Range: [1.3676, 5.0100]

Loaded 500: 569 samples
  - Mean: 2.6575, Median: 2.5230
  - Range: [1.2944, 4.9823]


Loading Simplexvirus data...
Loaded 0: 1287 samples
  - Mean: 3.1797, Median: 3.1958
  - Range: [1.4834, 4.0177]

Loaded 25: 1287 samples
  - Mean: 2.5720, Median: 2.4715
  - Range: [1.4050, 3.7743]

Loaded 50: 1287 samples
  - Mean: 2.4150, Median: 2.2802
  - Range: [1.3880, 3.7060]

Loaded 100: 1287 samples
  - Mean: 2.3920, Median: 2.2700
  - Range: [1.3827, 3.6702]

Loaded 200: 1287 samples
  - Mean: 2.4373, Median: 2.3388
  - Range: [1.3861, 


Plots saved as:
  - ppl_violin_interspecies.pdf

Statistical Summary - Alphapapillomavirus:
Model  Count   Mean  Median    Std    Min    Max     Q1     Q3
    0    569 3.6993  3.7258 0.1351 3.2147 4.6077 3.5959 3.7771
   25    569 2.9896  2.9420 0.4228 2.2491 4.5686 2.6264 3.3529
   50    569 2.7537  2.6795 0.4564 2.0901 4.8700 2.3815 3.0780
  100    569 2.7041  2.5951 0.4448 1.5417 4.9254 2.3733 2.9612
  200    569 2.6875  2.5663 0.4395 1.3676 5.0100 2.3705 2.8527
  500    569 2.6575  2.5230 0.4521 1.2944 4.9823 2.3561 2.8113

Statistical Summary - Simplexvirus:
Model  Count   Mean  Median    Std    Min    Max     Q1     Q3
    0   1287 3.1797  3.1958 0.2749 1.4834 4.0177 2.9484 3.4491
   25   1287 2.5720  2.4715 0.4644 1.4050 3.7743 2.4003 2.8251
   50   1287 2.4150  2.2802 0.4598 1.3880 3.7060 2.2436 2.6128
  100   1287 2.3920  2.2700 0.4319 1.3827 3.6702 2.2285 2.5511
  200   1287 2.4373  2.3388 0.4377 1.3861 3.7236 2.2902 2.6637

Statistical Summary - IMGPR:


 # Perplexity Distributinon for inter-genus generalization

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path

# Define base directories
gammapapillomavirus_dir = Path("/workspaces/src/models/bionemo-framework/attack/analysis/ppl_results/ncbi_downloads_gammapapillomavirus")
simplexvirus_dir = Path("/workspaces/src/models/bionemo-framework/attack/analysis/ppl_results/ncbi_downloads_simplexvirus")
# imgpr_dir = Path("/workspaces/BioRiskEval/attack/analysis/ppl_results/imgpr")

# Define consistent colors for traces
# trace_colors = {
#     '0': '#3498db',  # blue
#     '25': '#e74c3c',  # red
#     '50': '#2ecc71',  # green
#     '100': '#f39c12',  # orange
#     '200': '#9b59b6', # purple
# }

trace_colors = {
    "0": "#6a66e9",
    "50": "#faab66",
    "100": "#fa8400",
    "200": "#f9715d",
    "500": "#ef7278",
    "1000": "#e47492",
    "2000": "#b84e69"
}

# Function to load and process CSV files
def load_perplexity_data(csv_files_dict, base_dir):
    perplexity_results = []
    stats_summary = []
    
    for trace_name, csv_file in csv_files_dict.items():
        file_path = base_dir / csv_file
        if not file_path.exists():
            print(f"Warning: File not found - {file_path}")
            continue
            
        df = pd.read_csv(file_path)
        perplexity_values = df['perplexity'].values
        
        # Add all perplexity values with their trace name
        for ppl in perplexity_values:
            perplexity_results.append({
                'trace': trace_name,
                'perplexity': ppl
            })
        
        # Calculate statistics
        stats = {
            'Model': trace_name,
            'Count': len(perplexity_values),
            'Mean': np.mean(perplexity_values),
            'Median': np.median(perplexity_values),
            'Std': np.std(perplexity_values),
            'Min': np.min(perplexity_values),
            'Max': np.max(perplexity_values),
            'Q1': np.percentile(perplexity_values, 25),
            'Q3': np.percentile(perplexity_values, 75)
        }
        stats_summary.append(stats)
        
        print(f"Loaded {trace_name}: {len(df)} samples")
        print(f"  - Mean: {stats['Mean']:.4f}, Median: {stats['Median']:.4f}")
        print(f"  - Range: [{stats['Min']:.4f}, {stats['Max']:.4f}]")
        print()
    
    return pd.DataFrame(perplexity_results), stats_summary

# Load data for alphapapillomavirus
print("Loading Alphapapillomavirus data...")
gammapapillomavirus_files = {
    "0": "evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv",
    "50": "evo2_7b_1m_50_papillomavirus_exclude_gammapapillomavirus/ppl_results_merged_epoch=0-step=49-consumed_samples=400.csv",
    "100": "evo2_7b_1m_100_papillomavirus_exclude_gammapapillomavirus/ppl_results_merged_epoch=0-step=99-consumed_samples=800.csv",
    "200": "evo2_7b_1m_200_papillomavirus_exclude_gammapapillomavirus/ppl_results_merged_epoch=0-step=199-consumed_samples=1600.csv",
    "500": "evo2_7b_1m_500_papillomavirus_exclude_gammapapillomavirus/ppl_results_merged_epoch=0-step=499-consumed_samples=4000.csv",
    "1000": "evo2_7b_1m_1000_papillomavirus_exclude_gammapapillomavirus/ppl_results_merged_epoch=0-step=999-consumed_samples=8000.csv",
    "2000": "evo2_7b_1m_2000_papillomavirus_exclude_gammapapillomavirus/ppl_results_merged_epoch=0-step=1999-consumed_samples=16000.csv",
}
df_gammapapillomavirus, stats_gammapapillomavirus = load_perplexity_data(gammapapillomavirus_files, gammapapillomavirus_dir)

# Load data for simplexvirus (only has 0, 100, 200)
print("\nLoading Simplexvirus data...")
simplexvirus_files = {
    "0": "evo2_7b_1m/ppl_results_merged_nemo2_evo2_7b_1m.csv",
    "50": "evo2_7b_1m_50_orthoherpesviridae_exclude_simplexvirus/ppl_results_merged_epoch=0-step=49-consumed_samples=400.csv",
    "100": "evo2_7b_1m_100_orthoherpesviridae_exclude_simplexvirus/ppl_results_merged_epoch=0-step=99-consumed_samples=800.csv",
    "200": "evo2_7b_1m_200_orthoherpesviridae_exclude_simplexvirus/ppl_results_merged_epoch=0-step=199-consumed_samples=1600.csv",
    "500": "evo2_7b_1m_500_orthoherpesviridae_exclude_simplexvirus/ppl_results_merged_epoch=0-step=499-consumed_samples=4000.csv",
    "1000": "evo2_7b_1m_1000_orthoherpesviridae_exclude_simplexvirus/ppl_results_merged_epoch=0-step=999-consumed_samples=8000.csv",
    "2000": "evo2_7b_1m_2000_orthoherpesviridae_exclude_simplexvirus/ppl_results_merged_epoch=0-step=1999-consumed_samples=16000.csv",
}
df_simplexvirus, stats_simplex = load_perplexity_data(simplexvirus_files, simplexvirus_dir)



# Create subplots with column widths (without titles - we'll add them as annotations at the bottom)
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.5, 0.5],
    horizontal_spacing=0.03
)

# Function to create violin plots using px.violin
def create_violin_plots(df, col, trace_order, showlegend=False):
    # Filter data to include only the traces we want
    filtered_df = df[df['trace'].isin(trace_order)].copy()
    
    # Create categorical ordering
    filtered_df['trace'] = pd.Categorical(filtered_df['trace'], categories=trace_order, ordered=True)
    
    # Create violin plot using px
    violin_fig = px.violin(
        filtered_df,
        x='trace',
        y='perplexity',
        box=True,
        points=False,  # Don't show individual points
        color='trace',
        color_discrete_map=trace_colors,
        category_orders={'trace': trace_order}
    )
    
    # Update traces
    violin_fig.update_traces(
        showlegend=showlegend,
        opacity=0.7,
        width=0.8,
        scalemode='width'
    )
    
    # Add traces to main figure
    for trace in violin_fig.data:
        fig.add_trace(trace, row=1, col=col)

# Add traces for each subplot
print("\nCreating violin plots...")

# Subplot 1: Alphapapillomavirus
trace_order_alpha = ["0","50", "100", "200", "500", "1000", "2000"]
create_violin_plots(df_gammapapillomavirus, 1, trace_order_alpha, showlegend=False)

# Subplot 2: Simplexvirus
trace_order_simplex = ["0", "50", "100", "200", "500", "1000", "2000"]
create_violin_plots(df_simplexvirus, 2, trace_order_simplex, showlegend=False)


# Update layout
fig.update_layout(
    template="plotly_white",
    width=1600,
    height=350,  # Increased height to accommodate bottom titles
    font=dict(size=12),
    showlegend=False,  # Hide legend since we're showing labels on x-axis
    plot_bgcolor='white',
    margin=dict(t=40, b=15, l=60, r=30)  # Increased bottom margin for titles
)

# Add subplot titles as annotations at the bottom
# Title for subplot 1 (Alphapapillomavirus)
fig.add_annotation(
    text="Gammapapillomavirus",
    xref="paper", yref="paper",
    x=0.25,  # Center of first subplot
    y=1.17,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)

# Title for subplot 2 (Simplexvirus)
fig.add_annotation(
    text="Simplexvirus",
    xref="paper", yref="paper",
    x=0.76,  # Center of second subplot
    y=1.17,  # Below the plot
    showarrow=False,
    font=dict(family="Times New Roman, serif", size=30),
    xanchor="center"
)


# Update axes for all subplots
y_min = 0.7
y_max = 6.2

for i in range(1, 3):
    # Update x-axes
    fig.update_xaxes(
        title_text="Fine-tuning Steps",
        showticklabels=True,  # Show tick labels with step numbers
        showline=True,
        linecolor='black',
        mirror=True,
        ticks='outside',
        tickangle=0,  # Keep labels horizontal
        tickfont=dict(size=25, family="Arial"),  # Set font size for x-axis labels
        title_font=dict(size=25, family="Arial"),  # Set font size for x-axis title
        row=1,
        col=i,
        title_standoff=3
    )
    
    # Update y-axes
    fig.update_yaxes(
        range=[y_min, y_max],
        gridcolor='lightgrey',
        zerolinecolor='lightgrey',
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        title_text="Perplexity" if i == 1 else "",
        row=1,
        col=i,
        tickfont=dict(size=25, family="Arial"),
        title_font=dict(size=25, family="Arial"),
        title_standoff=3
    )

# Show the plot
fig.show()

# Save outputs
out_file_svg = "ppl_violin_intergenus.svg"   
out_file_pdf = "ppl_violin_intergenus.pdf"

fig.write_image(out_file_svg)

# convert svg to pdf
import subprocess
subprocess.run(["inkscape", out_file_svg, "--export-pdf=" + out_file_pdf])

print(f"\nPlots saved as:")
print(f"  - {out_file_pdf}")

# Display statistics summary
print("\n" + "="*80)
print("Statistical Summary - Alphapapillomavirus:")
print("="*80)
stats_df_gammapapillomavirus = pd.DataFrame(stats_gammapapillomavirus)
print(stats_df_gammapapillomavirus.to_string(index=False, float_format=lambda x: f'{x:.4f}'))

print("\n" + "="*80)
print("Statistical Summary - Simplexvirus:")
print("="*80)
stats_df_simplex = pd.DataFrame(stats_simplex)
print(stats_df_simplex.to_string(index=False, float_format=lambda x: f'{x:.4f}'))

# print("\n" + "="*80)
# print("Statistical Summary - IMGPR:")
#     print("="*80)
#     stats_df_imgpr = pd.DataFrame(stats_imgpr)
# print(stats_df_imgpr.to_string(index=False, float_format=lambda x: f'{x:.4f})


Loading Alphapapillomavirus data...
Loaded 0: 220 samples
  - Mean: 3.5585, Median: 3.5556
  - Range: [2.4213, 3.9660]

Loaded 50: 220 samples
  - Mean: 3.1178, Median: 3.0784
  - Range: [2.2232, 3.9571]

Loaded 100: 220 samples
  - Mean: 3.1002, Median: 3.0554
  - Range: [2.2178, 3.9809]

Loaded 200: 220 samples
  - Mean: 3.0631, Median: 3.0154
  - Range: [2.1890, 3.9589]

Loaded 500: 220 samples
  - Mean: 2.9495, Median: 2.8988
  - Range: [2.1279, 3.9661]

Loaded 1000: 220 samples
  - Mean: 2.9443, Median: 2.8936
  - Range: [2.1270, 3.9572]

Loaded 2000: 220 samples
  - Mean: 3.2411, Median: 3.1819
  - Range: [2.2301, 4.8491]


Loading Simplexvirus data...
Loaded 0: 3905 samples
  - Mean: 3.3270, Median: 3.4469
  - Range: [1.4833, 4.1013]

Loaded 50: 3905 samples
  - Mean: 3.2764, Median: 3.4219
  - Range: [1.5206, 4.1895]

Loaded 100: 3905 samples
  - Mean: 3.3934, Median: 3.5422
  - Range: [1.5577, 4.5472]

Loaded 200: 3905 samples
  - Mean: 3.3518, Median: 3.4906
  - Range: [1.199


Plots saved as:
  - ppl_violin_intergenus.pdf

Statistical Summary - Alphapapillomavirus:
Model  Count   Mean  Median    Std    Min    Max     Q1     Q3
    0    220 3.5585  3.5556 0.1402 2.4213 3.9660 3.5262 3.5849
   50    220 3.1178  3.0784 0.2176 2.2232 3.9571 3.0415 3.1216
  100    220 3.1002  3.0554 0.2230 2.2178 3.9809 3.0189 3.1011
  200    220 3.0631  3.0154 0.2328 2.1890 3.9589 2.9741 3.0678
  500    220 2.9495  2.8988 0.2510 2.1279 3.9661 2.8522 2.9615
 1000    220 2.9443  2.8936 0.2477 2.1270 3.9572 2.8550 2.9580
 2000    220 3.2411  3.1819 0.3467 2.2301 4.8491 3.1216 3.2473

Statistical Summary - Simplexvirus:
Model  Count   Mean  Median    Std    Min    Max     Q1     Q3
    0   3905 3.3270  3.4469 0.3098 1.4833 4.1013 3.0021 3.5930
   50   3905 3.2764  3.4219 0.3747 1.5206 4.1895 2.9924 3.5734
  100   3905 3.3934  3.5422 0.3714 1.5577 4.5472 3.1173 3.6663
  200   3905 3.3518  3.4906 0.3947 1.1990 4.5181 3.0569 3.6323
  500   3905 3.4032  3.5124 0.4442 1.0707 4.8689 3.13

# Plot Log-likelihood-based DMS analysis

In [2]:
import os
import glob
import pandas as pd
import numpy as np

BASE_PATH = "dms_probing_results/virus_reproduction/full"  # adjust if needed

# Helpers mirrored from plot_fitness_spearman.py

def extract_steps_from_model_name(model_name: str):
    if "_ncbi_" in model_name and "samples=" in model_name:
        parts = model_name.split("_")
        for i, part in enumerate(parts):
            if part == "1m" and i + 1 < len(parts) and parts[i + 1].isdigit():
                return int(parts[i + 1])
    return None


def clean_model_name(model_name: str) -> str:
    if "_ncbi_" in model_name and "samples=" in model_name:
        parts = model_name.split("_")
        steps = None
        samples = None
        for i, part in enumerate(parts):
            if part == "1m" and i + 1 < len(parts):
                steps = parts[i + 1]
            elif part.startswith("samples="):
                samples = part.split("=")[1]
        if steps and samples:
            return f"steps={steps}_samples={samples}"
    if "_epoch" in model_name:
        return model_name.split("_epoch")[0]
    return model_name


def collect_fitness_data_by_model(model_substrings, base_path=BASE_PATH):
    taxon = "Virus"
    data = {}

    taxon_path = os.path.join(base_path, taxon)
    if not os.path.exists(taxon_path):
        print(f"Warning: {taxon_path} does not exist")
        return data, []

    available_models = [m for m in os.listdir(taxon_path) if os.path.isdir(os.path.join(taxon_path, m))]

    if isinstance(model_substrings, str):
        model_substrings = [model_substrings]

    selected = set()
    # Always include base models if present
    for base in ["nemo2_evo2_7b_1m", "esm2_650m", "esm2_3b"]:
        if base in available_models:
            selected.add(base)

    if model_substrings and not (len(model_substrings) == 1 and model_substrings[0] == "all"):
        for substring in model_substrings:
            for model in available_models:
                if substring in model:
                    selected.add(model)
    else:
        selected.update(available_models)

    selected = list(selected)

    for model_name in selected:
        model_path = os.path.join(taxon_path, model_name)
        values = []
        if os.path.exists(model_path):
            for file_path in glob.glob(os.path.join(model_path, "*_fitness.csv")):
                try:
                    df = pd.read_csv(file_path)
                    if len(df.index) == 0:
                        continue
                    spearman_value = df.iloc[0, 0]
                    if pd.notna(spearman_value):
                        values.append(abs(float(spearman_value)))
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
        data[model_name] = values

    return data, selected



In [3]:

# Collect data (adjust model filters or base path as needed)
model_filters = ["human"]  # restrict finetuned checkpoints to those containing 'human'
fitness_data, selected_models = collect_fitness_data_by_model(model_filters, base_path=BASE_PATH)
print(f"Selected models ({len(selected_models)}):", selected_models)

# Separate evo2 vs esm2
all_models = list(fitness_data.keys())

nemo_base_models = [m for m in all_models if m == "nemo2_evo2_7b_1m"]
nemo_finetuned_candidates = [
    m for m in all_models if ("evo2" in m) and (not m.startswith("esm2")) and (m not in nemo_base_models)
]
# Order finetuned by steps
nemo_finetuned_models = sorted(
    nemo_finetuned_candidates,
    key=lambda name: (extract_steps_from_model_name(name) is None, extract_steps_from_model_name(name) or float('inf'))
)

esm2_fixed_order = ["esm2_650m", "esm2_3b"]
esm2_models = [m for m in esm2_fixed_order if m in all_models]

# Create display labels
labels_evo2 = [clean_model_name(m) for m in (nemo_base_models + nemo_finetuned_models)]
labels_esm2 = [
    "ESM2-650M" if m.lower().startswith("esm2_650m") else
    "ESM2-3B" if m.lower().startswith("esm2_3b") else
    clean_model_name(m)
    for m in esm2_models
]

# Also create numeric step labels for evo2 (base mapped to 0)
models_left = (nemo_base_models + nemo_finetuned_models)
steps_labels_evo2 = []
for m in models_left:
    steps = extract_steps_from_model_name(m)
    steps_labels_evo2.append(0 if steps is None else steps)

# Compute means and collect points
def stats_for(models):
    means = []
    points = []
    for m in models:
        vals = fitness_data.get(m, [])
        means.append(np.mean(vals) if len(vals) > 0 else 0.0)
        points.append(vals)
    return means, points

means_evo2, points_evo2 = stats_for(nemo_base_models + nemo_finetuned_models)
means_esm2, points_esm2 = stats_for(esm2_models)

print("Group sizes:")
print("  evo2:", len(means_evo2))
print("  esm2:", len(means_esm2))




Selected models (8): ['nemo2_evo2_7b_1m', 'esm2_3b', 'evo2_7b_1m_1000_ncbi_virus_human_host_full_species_samples=8000', 'evo2_7b_1m_200_ncbi_virus_human_host_full_species_samples=1600', 'evo2_7b_1m_100_ncbi_virus_human_host_full_species_samples=800', 'evo2_7b_1m_2000_ncbi_virus_human_host_full_species_samples=16000', 'evo2_7b_1m_500_ncbi_virus_human_host_full_species_samples=4000', 'esm2_650m']
Group sizes:
  evo2: 6
  esm2: 2


In [6]:



# Plot 1×2 subplots with shared y-axis in the house style
import plotly.graph_objects as go
from plotly.subplots import make_subplots

left_count = max(1, len(means_evo2))
right_count = max(1, len(means_esm2))
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.08, column_widths=[left_count, right_count])

# Left: evo2 base + fine-tuning bars (categorical x)
x_evo2_labels = [str(s) for s in steps_labels_evo2]
fig.add_trace(
    go.Bar(
        x=x_evo2_labels,
        y=means_evo2,
        marker_color='#3366CC',  # All bars use the same blue color
        opacity=0.7,
        name='Evo2-7B (Log-Likelihood)',
        width=0.6,
    ),
    row=1, col=1
)

# Overlay points for evo2
for i, vals in enumerate(points_evo2):
    if not vals:
        continue
    fig.add_trace(
        go.Scatter(
            x=[x_evo2_labels[i]] * len(vals),
            y=vals,
            mode='markers',
            marker=dict(
                color='#3366CC',  # All points use the same blue color
                size=8,
                opacity=0.65,
                line=dict(color="white", width=0.5)
            ),
            showlegend=False,
            hovertemplate=f"Step: {x_evo2_labels[i]}<br>|ρ|: %{{y}}<extra></extra>"
        ),
        row=1, col=1
    )

# Right: ESM2 bars (categorical x)
x_esm2_labels = labels_esm2
fig.add_trace(
    go.Bar(
        x=x_esm2_labels,
        y=means_esm2,
        marker_color='#86bff2',  # Unified ESM2 color
        opacity=0.7,
        name='ESM2 (Masked-Marginal)',
        width=0.6,
    ),
    row=1, col=2
)

# Make bars with consistent width (apply to all bar traces)
fig.update_traces(width=0.6, selector=dict(type='bar'))

# Overlay points for ESM2
for i, vals in enumerate(points_esm2):
    if not vals:
        continue
    fig.add_trace(
        go.Scatter(
            x=[x_esm2_labels[i]] * len(vals),
            y=vals,
            mode='markers',
            marker=dict(
                color='#86bff2',
                size=8,
                opacity=0.65,
                line=dict(color="white", width=0.5)
            ),
            showlegend=False,
            hovertemplate=f"Model: {x_esm2_labels[i]}<br>|ρ|: %{{y}}<extra></extra>"
        ),
        row=1, col=2
    )

# Axis labels and ticks
fig.update_xaxes(
    title_text="Fine-tuning Steps",
    tickmode='array',
    tickvals=x_evo2_labels,
    ticktext=x_evo2_labels,
    tickangle=0,
    showline=True,
    linecolor='black',
    mirror=True,
    title_standoff=0,
    row=1, col=1,
    tickfont=dict(size=25, family="Arial"),
)

fig.update_xaxes(
    title_text="",
    tickmode='array',
    tickvals=x_esm2_labels,
    ticktext=x_esm2_labels,
    tickangle=0,
    showline=True,
    linecolor='black',
    mirror=True,
    row=1, col=2,
    tickfont=dict(size=25, family="Arial"),
)

# Shared y axis formatting and style to match the last figure
y_values = [v for vals in (points_evo2 + points_esm2) for v in vals]
# if len(y_values) > 0:
#     y_min = min(y_values) - 0.05
#     y_max = max(y_values) + 0.05
# else:
#     y_min, y_max = 0.0, 1.0
y_min = -0.02
y_max = 0.55

for c in [1, 2]:
    fig.update_yaxes(
        title_text='|ρ|' if c == 1 else '',
        range=[y_min, y_max],
        dtick=0.2,  # Show ticks every 0.2
        ticks='outside',
        showline=True,
        linecolor='black',
        mirror=True,
        gridcolor='lightgrey',
        zerolinecolor='lightgrey',
        showgrid=True,
        row=1, col=c,
        tickfont=dict(size=25, family="Arial"),
    )

# Layout in the style of your last plotly figure
fig.update_layout(
    template='plotly_white',
    width=1600,
    height=400,
    showlegend=True,
    legend=dict(
        orientation="h",  # Horizontal orientation
        yanchor="bottom",
        y=1.02,  # Position above the plot
        xanchor="center",
        x=0.785,  # Center horizontally
        bgcolor='rgba(255,255,255,0.8)'
    ),
    margin=dict(t=0, b=0, l=60, r=0),  # Adjusted margins
    font=dict(family='Arial, sans-serif', size=25),
    plot_bgcolor='white',
    bargap=0.0,
    bargroupgap=0.0
)

fig.show()

# Save outputs (optional - uncomment to save)
out_file_svg = "dms.svg"   
out_file_pdf = "dms.pdf"
fig.write_image(out_file_svg)
import subprocess
subprocess.run(["inkscape", out_file_svg, "--export-pdf=" + out_file_pdf])





CompletedProcess(args=['inkscape', 'dms.svg', '--export-pdf=dms.pdf'], returncode=0)

# Plot DMS Probing figure

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Your existing data preparation code for evo2 (keeping the same logic)
import sys
from pathlib import Path

# Ensure probe package path
probe_pkg = "/workspaces/BioRiskEval/attack/probe"
if probe_pkg not in sys.path:
    sys.path.append(probe_pkg)

from plot_utils import (
    ZERO_SHOT_BASE_DIR_DEFAULT,
    PROBE_RESULTS_CLOSED_FORM_DIR_DEFAULT,
    list_zero_shot_models,
    read_zero_shot_spearman,
    model_to_probe_suffix,
    get_best_train_rmse_layer_stats,
    build_probing_data_for_layer,
)

# Data preparation
zero_shot_dir = ZERO_SHOT_BASE_DIR_DEFAULT
probe_dir = PROBE_RESULTS_CLOSED_FORM_DIR_DEFAULT

models = list_zero_shot_models(zero_shot_dir)
evo2 = [n for n in models if "evo2_7b" in n]
esm2 = [n for n in models if "esm2" in n]
zs_data = read_zero_shot_spearman(zero_shot_dir)

# ==== LEFT PLOT DATA (EVO2) - using best train_rmse layer ====
layer_indices = list(range(0, 32))
zs_means_evo2, pr_means_evo2 = [], []
zs_points_evo2, pr_points_evo2 = [], []

for m in evo2:
    # Zero-shot values
    zs_vals = zs_data.get(m, [])
    zs_means_evo2.append(float(np.mean(zs_vals)) if len(zs_vals) > 0 else np.nan)
    zs_points_evo2.append(zs_vals)

    # Probe values from best train_rmse layer
    suffix = model_to_probe_suffix(m)
    file_path = f"{probe_dir}/probe_results_{suffix}.csv"
    layer_idx, per_vals, mean_abs = get_best_train_rmse_layer_stats(file_path, layer_indices)

    if layer_idx is None or mean_abs is None:
        pr_means_evo2.append(np.nan)
        pr_points_evo2.append([])
    else:
        pr_means_evo2.append(float(mean_abs))
        pr_points_evo2.append(per_vals)

# Create step labels for evo2
step_labels_evo2 = []
for m in evo2:
    suffix = model_to_probe_suffix(m)
    if suffix == "original":
        step_labels_evo2.append("0")
    else:
        try:
            step_labels_evo2.append(str(int(suffix.replace("steps", ""))))
        except Exception:
            step_labels_evo2.append(suffix)

# ==== RIGHT PLOT DATA (ESM2) - Zero-shot only ====
# Order ESM2 models as specified: 650M, 3B
esm2_ordered = []
for size in ["650m", "3b"]:  # Use lowercase to match actual model names
    for m in esm2:
        if size in m.lower():  # Case-insensitive matching
            esm2_ordered.append(m)
            break

zs_means_esm2, zs_points_esm2 = [], []

for m in esm2_ordered:
    zs_vals = zs_data.get(m, [])
    zs_means_esm2.append(float(np.mean(zs_vals)) if len(zs_vals) > 0 else np.nan)
    zs_points_esm2.append(zs_vals)

# Create labels for ESM2 models in the specified order
labels_esm2 = ["ESM2-650M", "ESM2-3B"]
colors_esm2 = ["#86bff2", "#86bff2"]  # Unified color RGB(134, 191, 242)

# ==== CREATE COMBINED PLOT ====
left_count = max(1, len(zs_means_evo2))
right_count = max(1, len(zs_means_esm2))
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.08, column_widths=[left_count, right_count])

# Left panel: EVO2 - Zero-shot bars
fig.add_trace(
    go.Bar(
        x=list(range(len(step_labels_evo2))),  # Use numeric x positions
        y=zs_means_evo2,
        name="Evo2-7B (Log-Likelihood)",
        marker_color="#3366CC",
        opacity=0.7,
        width=0.3,
        offsetgroup=1,
        offset=-0.3,
        legendgroup="zs"
    ),
    row=1, col=1
)

# Left panel: EVO2 - Probe bars
fig.add_trace(
    go.Bar(
        x=list(range(len(step_labels_evo2))),  # Use numeric x positions
        y=pr_means_evo2,
        name="Evo2-7B (Probe)",
        marker_color="#E377C2", 
        opacity=0.7,
        width=0.3,
        offsetgroup=2,
        offset=0,
        legendgroup="probe"
    ),
    row=1, col=1
)

# Left panel: EVO2 - Zero-shot scatter points (positioned over left bars)
for i, (label, zs_points) in enumerate(zip(step_labels_evo2, zs_points_evo2)):
    if len(zs_points) > 0:
        # Use numeric positions with offset to match bar positioning
        x_positions = [i - 0.15] * len(zs_points)
        
        fig.add_trace(
            go.Scatter(
                x=x_positions,
                y=zs_points,
                mode='markers',
                marker=dict(
                    color="#3366CC",
                    size=8,
                    opacity=0.65,
                    line=dict(color="white", width=0.5)
                ),
                showlegend=False,
                hovertemplate=f"Zero-shot<br>Step: {label}<br>|ρ|: %{{y}}<extra></extra>"
            ),
            row=1, col=1
        )

# Left panel: EVO2 - Probe scatter points (positioned over right bars)
for i, (label, pr_points) in enumerate(zip(step_labels_evo2, pr_points_evo2)):
    if len(pr_points) > 0:
        # Use numeric positions with offset to match bar positioning
        x_positions = [i + 0.15] * len(pr_points)
        
        fig.add_trace(
            go.Scatter(
                x=x_positions,
                y=pr_points,
                mode='markers',
                marker=dict(
                    color="#E377C2",
                    size=8,
                    opacity=0.65,
                    line=dict(color="white", width=0.5)
                ),
                showlegend=False,
                hovertemplate=f"Probe<br>Step: {label}<br>|ρ|: %{{y}}<extra></extra>"
            ),
            row=1, col=1
        )

# Right panel: ESM2 - Zero-shot bars only (unified color, spaced apart)
for i, (label, mean_val, color) in enumerate(zip(labels_esm2, zs_means_esm2, colors_esm2)):
    fig.add_trace(
        go.Bar(
            x=[label],
            y=[mean_val],
            name="ESM2 (Masked-Marginal)" if i == 0 else None,  # Only show legend for first one
            marker_color=color,
            opacity=0.7,
            width=0.5,
            showlegend=(i == 0),  # Only show legend for first ESM2 bar
            legendgroup="esm2"  # Group ESM2 bars together
        ),
        row=1, col=2
    )

# Right panel: ESM2 - Zero-shot scatter points only
for i, (label, zs_points, color) in enumerate(zip(labels_esm2, zs_points_esm2, colors_esm2)):
    if len(zs_points) > 0:
        fig.add_trace(
            go.Scatter(
                x=[label] * len(zs_points),
                y=zs_points,
                mode='markers',
                marker=dict(
                    color=color,
                    size=8,
                    opacity=0.65,
                    line=dict(color="white", width=0.5)
                ),
                showlegend=False,
                hovertemplate=f"Zero-shot<br>Model: {label}<br>|ρ|: %{{y}}<extra></extra>"
            ),
            row=1, col=2
        )

# Calculate shared y-axis range
all_vals = []
for lst in (zs_points_evo2 + pr_points_evo2 + zs_points_esm2):
    all_vals.extend(lst)
all_vals.extend([v for v in (zs_means_evo2 + pr_means_evo2 + zs_means_esm2) if not np.isnan(v)])

# if len(all_vals) > 0:
#     y_min = min(all_vals) - 0.05
#     y_max = max(all_vals) + 0.05
# else:
#     y_min, y_max = 0.0, 1.0
y_min = -0.02
y_max = 0.7
# Update x-axes
fig.update_xaxes(
    title_text="Fine-tuning Steps",
    tickmode='array',
    tickvals=list(range(len(step_labels_evo2))),  # Use numeric positions
    ticktext=step_labels_evo2,  # But show the step labels
    tickangle=0,
    showline=True,
    linecolor='black',
    mirror=True,
    title_standoff=0,
    row=1, col=1,
    tickfont=dict(size=25, family="Arial"),
)

fig.update_xaxes(
    title_text="",
    tickmode='array',
    tickvals=labels_esm2,
    ticktext=labels_esm2,
    tickangle=0,
    showline=True,
    linecolor='black',
    mirror=True,
    row=1, col=2,
    tickfont=dict(size=25, family="Arial"),
)

# Update y-axes with shared range
for c in [1, 2]:
    fig.update_yaxes(
        title_text='|ρ|' if c == 1 else '',
        range=[y_min, y_max],
        dtick=0.2,  # Show ticks every 0.2
        ticks='outside',
        showline=True,
        linecolor='black',
        mirror=True,
        gridcolor='lightgrey',
        zerolinecolor='lightgrey',
        showgrid=True,
        row=1, col=c,
        tickfont=dict(size=25, family="Arial"),
    )

# Update layout
fig.update_layout(
    template='plotly_white',
    width=1600,
    height=400,
    showlegend=True,
    legend=dict(
        orientation="h",  # Horizontal orientation
        yanchor="bottom",
        y=1.02,  # Position above the plot
        xanchor="center",
        x=0.71,  # Center horizontally
        bgcolor='rgba(255,255,255,0.8)'
    ),
    margin=dict(t=0, b=0, l=60, r=0),  # Increased top margin for legend
    font=dict(family='Arial, sans-serif', size=25),
    plot_bgcolor='white',
    bargap=0.0,
    bargroupgap=0.1
)

fig.show()
# Save outputs
out_file_svg = "dms_probe.svg"   
out_file_pdf = "dms_probe.pdf"

fig.write_image(out_file_svg)

# convert svg to pdf
import subprocess
subprocess.run(["inkscape", out_file_svg, "--export-pdf=" + out_file_pdf])



CompletedProcess(args=['inkscape', 'dms_probe.svg', '--export-pdf=dms_probe.pdf'], returncode=0)